<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/SMA_Short_VIX_Adjust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SMA_Short_VIX_Adjust
---
This notebook analyzes the historical price data of the ticker to evaluate the impact of **Simple Moving Average (SMA)** sign changes on asset returns.

* It begins by downloading historical data via the `yfinance` library and cleaning the resulting DataFrame into a flat table format.
* A custom function calculates the SMA for a given window and identifies "sign changes," which occur when the closing price crosses above or below the moving average.
* The analysis specifically filters for instances where the price drops below the SMA (sign change to -1) and calculates the subsequent return between those events.
* The script iterates through a range of lookback periods (from 3 to 21) to aggregate these returns by year, allowing for a comparison of how different SMA lengths perform over time.
* The final output is a concatenated DataFrame that summarizes the total yearly returns triggered by these SMA crossovers for each lookback value.

---

In [73]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define a ticker and a date range for your data
ticker = 'UVXY'
start_date = '2012-01-01'
end_date = '2025-12-25'
window = 5
lookback_range = [3,21]
vix_limit = 18

# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
data = data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
data = data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
data.insert(1, 'Ticker', ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2012-01-03,UVXY,1.632000e+11,1.675500e+11,1.612500e+11,1.656000e+11,0
1,2012-01-04,UVXY,1.569000e+11,1.705500e+11,1.564500e+11,1.678500e+11,0
2,2012-01-05,UVXY,1.497000e+11,1.642500e+11,1.492500e+11,1.611000e+11,0
3,2012-01-06,UVXY,1.449000e+11,1.530000e+11,1.419000e+11,1.471500e+11,0
4,2012-01-09,UVXY,1.405500e+11,1.446000e+11,1.390500e+11,1.422000e+11,0
...,...,...,...,...,...,...,...
3511,2025-12-18,UVXY,4.108000e+01,4.255000e+01,4.060000e+01,4.165000e+01,10864500
3512,2025-12-19,UVXY,3.866000e+01,4.045000e+01,3.853000e+01,4.040000e+01,6814300
3513,2025-12-22,UVXY,3.680000e+01,3.784000e+01,3.667000e+01,3.754000e+01,5383700
3514,2025-12-23,UVXY,3.698000e+01,3.716000e+01,3.661000e+01,3.708000e+01,4483100


In [74]:
# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
vix_ticker = '^VIX'
vix_data = yf.download(vix_ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
vix_data = vix_data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
vix_data = vix_data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
vix_data.insert(1, 'Ticker', vix_ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(vix_data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2012-01-03,^VIX,22.969999,23.100000,22.540001,22.950001,0
1,2012-01-04,^VIX,22.219999,23.730000,22.219999,23.440001,0
2,2012-01-05,^VIX,21.480000,23.090000,21.340000,22.750000,0
3,2012-01-06,^VIX,20.629999,21.719999,20.580000,21.240000,0
4,2012-01-09,^VIX,21.070000,21.780001,21.000000,21.670000,0
...,...,...,...,...,...,...,...
3511,2025-12-18,^VIX,16.870001,17.680000,15.930000,17.610001,0
3512,2025-12-19,^VIX,14.910000,16.530001,14.910000,16.309999,0
3513,2025-12-22,^VIX,14.080000,15.260000,14.030000,15.160000,0
3514,2025-12-23,^VIX,14.000000,14.450000,13.640000,14.090000,0


In [75]:
# 1. Identify dates present in 'data' but NOT in 'vix_data'
missing_vix_dates = data[~data['Date'].isin(vix_data['Date'])]

# 2. Check the results
if not missing_vix_dates.empty:
    print(f"⚠️ Found {len(missing_vix_dates)} days in {ticker} that are missing from VIX data:")
    # Display the specific dates and the ticker values for those days
    display(missing_vix_dates[['Date', 'Ticker', 'Close']])
else:
    print(f"✅ All {len(data)} trading days in {ticker} are present in the VIX dataset.")

# 3. Optional: Detailed check for the first few missing dates (if any)
# This helps identify if it's a specific period or random holidays
if not missing_vix_dates.empty:
    print("\nFirst 5 missing dates:")
    print(missing_vix_dates['Date'].head().tolist())

✅ All 3516 trading days in UVXY are present in the VIX dataset.


In [76]:
def calculate_sma(df, lookback):
    """
    Calculates a simple moving average for a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'close' column.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: The DataFrame with a new column for the moving average.
    """
    # Create a copy to avoid modifying the original DataFrame
    df_sma = df.copy()

    # Calculate the simple moving average
    df_sma['SMA'] = df_sma['Close'].rolling(window=lookback).mean()

    # Calculate the difference between the SMA and the Close price
    df_sma['SMA_sign'] = np.sign(df_sma['Close'] - df_sma['SMA'])

    return df_sma



In [79]:
    # 1. Calculate SMA and signs for every day
    df_with_sma = calculate_sma(df=data.copy(), lookback=7)

    # 2. Merge VIX data into the main dataframe based on Date
    # We use 'left' join to keep all price dates and pull in the corresponding VIX close
    vix_subset = vix_data[['Date', 'Close']].rename(columns={'Close': 'VIX_Close'})
    df_with_sma = pd.merge(df_with_sma, vix_subset, on='Date', how='left')

    # 3. Identify ALL sign changes (crosses up AND crosses down)
    previous_sign = df_with_sma['SMA_sign'].shift(1)
    all_crosses_mask = (df_with_sma['SMA_sign'] != previous_sign) & (~previous_sign.isna())

    # Create a DataFrame of every signal event
    all_signals = df_with_sma[all_crosses_mask].copy()

    display(all_signals)

Price,Date,Ticker,Close,High,Low,Open,Volume,SMA,SMA_sign,VIX_Close
8,2012-01-13,UVXY,1.422000e+11,1.503000e+11,1.410000e+11,1.416000e+11,0,1.408071e+11,1.0,20.910000
10,2012-01-18,UVXY,1.302000e+11,1.414500e+11,1.296000e+11,1.383000e+11,0,1.373357e+11,-1.0,20.889999
26,2012-02-09,UVXY,9.300000e+10,9.315000e+10,8.415000e+10,8.550000e+10,0,8.685000e+10,1.0,18.629999
32,2012-02-17,UVXY,9.900000e+10,1.023000e+11,9.510000e+10,9.690000e+10,0,1.000714e+11,-1.0,17.780001
43,2012-03-06,UVXY,9.195000e+10,9.345000e+10,8.655000e+10,8.985000e+10,0,8.380714e+10,1.0,20.870001
...,...,...,...,...,...,...,...,...,...,...
3484,2025-11-10,UVXY,5.035000e+01,5.205000e+01,5.005000e+01,5.150000e+01,6431460,5.336429e+01,-1.0,17.600000
3487,2025-11-13,UVXY,5.575000e+01,5.700000e+01,5.070000e+01,5.145000e+01,11096520,5.273571e+01,1.0,20.000000
3494,2025-11-24,UVXY,5.385000e+01,5.933000e+01,5.385000e+01,5.900000e+01,9691400,5.902571e+01,-1.0,20.520000
3510,2025-12-17,UVXY,4.325000e+01,4.339000e+01,4.116000e+01,4.178000e+01,11122000,4.301571e+01,1.0,17.620001


In [64]:
def analyze_sma_changes(df, vix_df, lookback, vix_threshold):
    """
    Analyzes the impact of Simple Moving Average bearish sign changes on price differences.

    Args:
        df (pd.DataFrame): The input DataFrame with historical price data.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: A DataFrame containing the sum of Next_Close_Diff at SMA sign changes by year,
                      including the lookback value.
    """

    # 1. Calculate SMA and signs for every day
    df_with_sma = calculate_sma(df=df.copy(), lookback=lookback)

    # 2. Merge VIX data into the main dataframe based on Date
    # We use 'left' join to keep all price dates and pull in the corresponding VIX close
    vix_subset = vix_df[['Date', 'Close']].rename(columns={'Close': 'VIX_Close'})
    df_with_sma = pd.merge(df_with_sma, vix_subset, on='Date', how='left')

    # 3. Identify ALL sign changes (crosses up AND crosses down)
    previous_sign = df_with_sma['SMA_sign'].shift(1)
    all_crosses_mask = (df_with_sma['SMA_sign'] != previous_sign) & (~previous_sign.isna())

    # Create a DataFrame of every signal event
    all_signals = df_with_sma[all_crosses_mask].copy()

    # 4. Calculate Bearish Return: (Entry Price - Exit Price) / Entry Price
    # Because all_signals contains the NEXT signal (the Bullish cross),
    # shift(-1) now correctly points to the "Exit".
    all_signals.loc[:, 'Next_Close_Return'] = (
        (all_signals['Close'] - all_signals['Close'].shift(-1)) / all_signals['Close']
    )

    # 5. FILTER FOR BEARISH ENTRIES + VIX THRESHOLD
    # - SMA_sign == -1 (Price crossed below SMA)
    # - VIX_Close > vix_threshold (Volatility is high enough)
    bearish_trades = all_signals[
        (all_signals['SMA_sign'] == -1) &
        (all_signals['VIX_Close'] > vix_threshold)
    ].copy()

    # A trade is successful if the return is positive
    bearish_trades['Is_Success'] = bearish_trades['Next_Close_Return'] > 0

    # 6. Clean up and Aggregate
    bearish_trades['Date'] = pd.to_datetime(bearish_trades['Date'])
    bearish_trades['Year'] = bearish_trades['Date'].dt.year

    # Update GroupBy to aggregate Sum, Count (Total Trades), and Sum of Successes (Number of Wins)
    yearly_results = bearish_trades.groupby('Year').agg(
        Total_Return=('Next_Close_Return', 'sum'),
        Trade_Count=('Next_Close_Return', 'count'),
        Success_Count=('Is_Success', 'sum')
    ).reset_index()

    # yearly_results = bearish_trades.groupby('Year')['Next_Close_Return'].sum().reset_index()
    yearly_results['Lookback'] = lookback
    yearly_results['VIX_Threshold'] = vix_threshold

    # Optional: Add Win Rate percentage
    yearly_results['Win_Rate'] = yearly_results['Success_Count'] / yearly_results['Trade_Count']

    return yearly_results[['Year', 'Lookback', 'VIX_Threshold', 'Total_Return', 'Trade_Count', 'Success_Count', 'Win_Rate']]

In [65]:
# Initialize an empty list to store the results from each lookback value
results_list = []

# Iterate through each value in the specified range
for lookback_value in range(lookback_range[0], lookback_range[1] + 1):
    # Calculate SMA changes for the current lookback value
    #df_sma_result = analyze_sma_changes(df=data.copy(), lookback=lookback_value)
    df_sma_result = analyze_sma_changes(df=data.copy(), vix_df=vix_data, lookback=lookback_value, vix_threshold=vix_limit)
     # Append the result to the list
    results_list.append(df_sma_result)

# Concatenate all the DataFrames in the list into a single DataFrame
all_sma_results = pd.concat(results_list, ignore_index=True)

# Print the resulting DataFrame
print("DataFrame with Simple Moving Average analysis for different lookback values:")
display(all_sma_results)

DataFrame with Simple Moving Average analysis for different lookback values:


,Year,Lookback,VIX_Threshold,Total_Return,Trade_Count,Success_Count,Win_Rate
0,2012,3,18,1.518477,18,10,0.555556
1,2013,3,18,0.604204,2,2,1.000000
2,2014,3,18,0.329355,2,2,1.000000
3,2015,3,18,0.585802,12,8,0.666667
4,2016,3,18,0.441661,10,5,0.500000
...,...,...,...,...,...,...,...
217,2021,21,18,0.275146,14,7,0.500000
218,2022,21,18,-0.193160,14,4,0.285714
219,2023,21,18,0.496538,5,3,0.600000
220,2024,21,18,0.071249,5,2,0.400000


In [66]:
# 1. Pivot the DataFrame using 'Trade_Count' as the values
pivoted_trade_results = all_sma_results.pivot(
    index='Year',
    columns='Lookback',
    values='Trade_Count'
)

# 2. Fill missing values with 0 AND convert to integer
# .astype(int) ensures the counts are displayed as whole numbers (e.g., 5 instead of 5.0)
pivoted_trade_results = pivoted_trade_results.fillna(0).astype(int)

# 3. Display the results
print("Pivoted DataFrame: Trade Count by Year and Lookback:")

display(pivoted_trade_results)

Pivoted DataFrame: Trade Count by Year and Lookback:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2012,18,16,11,10,10,8,7,7,7,9,10,9,7,7,8,8,7,8,8
2013,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2014,2,2,2,2,2,2,2,1,1,1,1,0,0,0,0,0,0,0,0
2015,12,9,7,8,5,5,4,5,5,5,4,4,3,3,3,3,3,3,2
2016,10,9,9,8,6,6,6,6,7,7,6,6,5,4,4,4,5,5,6
2018,17,14,11,12,10,6,6,7,7,7,7,7,8,8,7,5,5,6,6
2019,8,4,3,6,2,3,5,4,5,3,1,1,1,1,1,1,1,1,1
2020,36,32,28,25,21,20,14,14,13,14,14,14,15,15,13,13,12,12,11
2021,30,27,25,22,17,17,17,18,19,19,19,18,17,18,17,17,16,14,14


In [67]:
# 1. Pivot the DataFrame using 'success_rate' as the values
pivoted_success_results = all_sma_results.pivot(
    index='Year',
    columns='Lookback',
    values='Win_Rate'
)

# 2. Fill missing values (years with no trades) with 0
pivoted_success_results = pivoted_success_results.fillna(0)

# 3. Display the results
print("Pivoted DataFrame: Success Rate (Wins / Total Trades) by Year and Lookback:")
# Optional: format as percentages for better readability
display(pivoted_success_results.style.format("{:.2%}"))

Pivoted DataFrame: Success Rate (Wins / Total Trades) by Year and Lookback:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2012,55.56%,50.00%,72.73%,70.00%,60.00%,75.00%,71.43%,71.43%,71.43%,55.56%,50.00%,55.56%,57.14%,57.14%,50.00%,37.50%,42.86%,50.00%,50.00%
2013,100.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2014,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,50.00%,100.00%,100.00%,100.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2015,66.67%,44.44%,42.86%,37.50%,60.00%,60.00%,75.00%,60.00%,60.00%,40.00%,50.00%,50.00%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,50.00%
2016,50.00%,55.56%,44.44%,50.00%,66.67%,50.00%,50.00%,50.00%,42.86%,42.86%,50.00%,50.00%,60.00%,75.00%,75.00%,75.00%,60.00%,60.00%,50.00%
2018,35.29%,35.71%,45.45%,33.33%,50.00%,66.67%,50.00%,42.86%,57.14%,57.14%,14.29%,14.29%,25.00%,25.00%,28.57%,40.00%,60.00%,50.00%,50.00%
2019,37.50%,25.00%,66.67%,66.67%,50.00%,33.33%,40.00%,25.00%,40.00%,66.67%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2020,52.78%,53.12%,50.00%,56.00%,52.38%,45.00%,50.00%,50.00%,53.85%,50.00%,42.86%,50.00%,53.33%,53.33%,53.85%,53.85%,50.00%,41.67%,45.45%
2021,63.33%,51.85%,40.00%,40.91%,41.18%,41.18%,41.18%,38.89%,36.84%,36.84%,36.84%,38.89%,41.18%,38.89%,41.18%,41.18%,43.75%,50.00%,50.00%


In [68]:
# Pivot the DataFrame
pivoted_sma_results = all_sma_results.pivot(index='Year', columns='Lookback', values='Total_Return')

pivoted_sma_results = pivoted_sma_results.fillna(0)

# Display the pivoted DataFrame
print("Pivoted DataFrame with Lookback as columns, Year as rows, and Total_Return as values:")
display(pivoted_sma_results)

Pivoted DataFrame with Lookback as columns, Year as rows, and Total_Return as values:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2012,1.518477,1.462070,1.363585,1.278909,1.809015,1.662597,1.514705,1.799580,1.799580,1.532919,1.479221,1.567675,1.240885,1.240885,1.174492,1.167856,1.182219,1.115338,0.995366
2013,0.604204,0.390902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014,0.329355,0.630764,0.508292,0.597377,0.500460,0.502147,0.357185,0.368850,0.368850,0.380939,0.380939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015,0.585802,0.291696,0.097449,0.099459,0.535460,0.575357,0.660838,0.468062,0.424807,0.289104,0.332204,0.332204,0.215787,0.215787,0.216119,0.051120,-0.038766,-0.038766,0.154010
2016,0.441661,0.938740,0.584311,0.795673,1.094197,1.153251,1.069644,1.083293,0.895151,1.065026,1.146798,1.146145,1.403133,1.483326,1.483326,1.366832,1.459873,1.844973,1.834931
2018,-0.458142,-0.118320,0.181737,-0.167380,0.243398,0.355070,0.133371,-0.058239,-0.065126,-0.169368,-0.073095,-0.073095,-0.266822,-0.164944,0.012119,0.202303,0.177702,0.039680,0.054229
2019,0.235455,0.077292,0.245988,0.738080,0.238133,0.223514,0.277395,0.016837,0.277395,0.318354,0.395295,0.395295,0.395295,0.359668,0.359668,0.359668,0.359668,0.359668,0.336293
2020,0.667361,0.946321,0.944927,0.638175,0.676449,0.257814,0.660689,0.614415,0.711001,0.416517,0.264488,0.248707,0.106742,0.106742,0.389221,0.363408,0.439383,0.337248,0.419053
2021,0.861419,0.483054,0.050046,0.338119,0.690326,0.258527,0.215780,0.164444,0.111512,0.187139,0.227648,0.202243,-0.129734,-0.048742,0.029788,0.029788,0.083084,0.245133,0.275146


In [69]:
# Ensure 'Date' column in the initial 'data' DataFrame is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Extract the year from the 'Date' column
data['Year'] = data['Date'].dt.year

# Group by year and get the first and last close prices
yearly_price_change = data.groupby('Year')['Close'].agg(['first', 'last'])

# Calculate the difference between the last and first close price for each year
yearly_price_change['Yearly_Return'] = (yearly_price_change['first'] - yearly_price_change['last']) / yearly_price_change['first']

# Drop the 'first' and 'last' columns
yearly_price_change = yearly_price_change.drop(columns=['first', 'last'])

# Display the result
print("Difference between the last and first close price of each year:")
display(yearly_price_change)

Difference between the last and first close price of each year:


,Yearly_Return
Year,
2012,0.967984
2013,0.895582
2014,0.639478
2015,0.766954
2016,0.945244
2017,0.931933
2018,-0.729735
2019,0.834574
2020,0.122012


In [70]:
# Join the yearly_price_change DataFrame with the yearly_next_close_diff_sum Series on the 'Year' index
comparison_df = yearly_price_change.join(pivoted_sma_results)

# Display the combined DataFrame, excluding the 'first' and 'last' columns
display(comparison_df)

,Yearly_Return,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,,
2012,0.967984,1.518477,1.462070,1.363585,1.278909,1.809015,1.662597,1.514705,1.799580,1.799580,1.532919,1.479221,1.567675,1.240885,1.240885,1.174492,1.167856,1.182219,1.115338,0.995366
2013,0.895582,0.604204,0.390902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014,0.639478,0.329355,0.630764,0.508292,0.597377,0.500460,0.502147,0.357185,0.368850,0.368850,0.380939,0.380939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015,0.766954,0.585802,0.291696,0.097449,0.099459,0.535460,0.575357,0.660838,0.468062,0.424807,0.289104,0.332204,0.332204,0.215787,0.215787,0.216119,0.051120,-0.038766,-0.038766,0.154010
2016,0.945244,0.441661,0.938740,0.584311,0.795673,1.094197,1.153251,1.069644,1.083293,0.895151,1.065026,1.146798,1.146145,1.403133,1.483326,1.483326,1.366832,1.459873,1.844973,1.834931
2017,0.931933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,-0.729735,-0.458142,-0.118320,0.181737,-0.167380,0.243398,0.355070,0.133371,-0.058239,-0.065126,-0.169368,-0.073095,-0.073095,-0.266822,-0.164944,0.012119,0.202303,0.177702,0.039680,0.054229
2019,0.834574,0.235455,0.077292,0.245988,0.738080,0.238133,0.223514,0.277395,0.016837,0.277395,0.318354,0.395295,0.395295,0.395295,0.359668,0.359668,0.359668,0.359668,0.359668,0.336293
2020,0.122012,0.667361,0.946321,0.944927,0.638175,0.676449,0.257814,0.660689,0.614415,0.711001,0.416517,0.264488,0.248707,0.106742,0.106742,0.389221,0.363408,0.439383,0.337248,0.419053


In [71]:
# Calculate the sum and standard deviation of each column in the comparison_df
grand_totals = comparison_df.sum()
standard_deviations = comparison_df.std()

# Calculate the Mean Absolute Deviation for each column
mean_absolute_deviations = comparison_df.apply(lambda x: (x - x.mean()).abs().mean())

# Combine the metrics into a single DataFrame for display
summary_df = pd.DataFrame({
    'Grand Total': grand_totals,
    'Standard Deviation': standard_deviations,
    'Mean Absolute Deviation': mean_absolute_deviations
})

# Display the summary DataFrame
print("Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:")
display(summary_df)

Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:


,Grand Total,Standard Deviation,Mean Absolute Deviation
Yearly_Return,8.729193,0.458569,0.310603
3,5.849811,0.507413,0.330248
4,6.185491,0.457312,0.348275
5,5.712770,0.400458,0.316462
6,6.282242,0.387237,0.308948
7,7.728468,0.457032,0.293635
8,6.775183,0.439847,0.295206
9,5.978051,0.428016,0.317919
10,5.029008,0.539891,0.377560
11,5.170757,0.514196,0.356008
